# File System Tool

The `uwtools` API's `fs` module provides functions to copy and link files as well as create directories. 

For more information, please see the <a href="https://uwtools.readthedocs.io/en/main/sections/user_guide/api/fs.html">uwtools.api.fs</a> Read the Docs page.

## Table of Contents

* [Copying Files](#Copying-Files)
    * [Failing to copy](#Failing-to-copy)
    * [Using the `keys` parameter](#Using-the-keys-parameter)
    * [Using the `Copier` class](#Using-the-Copier-class)
* [Linking Files](#Linking-files)
    * [Failing to link](#Failing-to-link)
    * [Using the `keys` parameter](#Using-the-keys-parameter-)
    * [Using the `Linker` class](#Using-the-Linker-class)
* [Creating directories](#Creating-directories)
    * [Using the `keys` parameter](#Using-the-keys-parameter--)
    * [Using the `MakeDirs` class](#Using-the-MakeDirs-class)

In [1]:
from pathlib import Path
from shutil import rmtree
from uwtools.api import fs
from uwtools.api.logging import use_uwtools_logger

use_uwtools_logger()

## Copying Files

The `copy()` function copies files, automatically creating parent directories as needed.

In [2]:
help(fs.copy)

Help on function copy in module uwtools.api.fs:

copy(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False, stdin_ok: bool = False) -> bool
    Copy files.

    :param config: YAML-file path, or ``dict`` (read ``stdin`` if missing or ``None``).
    :param target_dir: Path to target directory.
    :param cycle: A datetime object to make available for use in the config.
    :param leadtime: A timedelta object to make available for use in the config.
    :param keys: YAML keys leading to file dst/src block.
    :param dry_run: Do not copy files.
    :param stdin_ok: OK to read from ``stdin``?
    :return: ``True`` if all copies were created.



Files to be copied are specified by a mapping from keys destination-pathname keys to source-pathname values, either in a YAML file or a a Python ``dict``.

In [3]:
%%bash
cat fixtures/fs/copy-config.yaml

file1-copy.nml: fixtures/fs/file1.nml
data/file2-copy.txt: fixtures/fs/file2.txt
data/file3-copy.csv: fixtures/fs/file3.csv


With these instructions, `copy()` creates a copy of each given file with the given name and in the given subdirectory. Copies are created in the directory indicated by `target_dir`. Paths can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Any directories in the targeted paths for copying will be created if they don't already exist. `True` is returned upon a successful copy.

In [4]:
rmtree("tmp/copy-target", ignore_errors=True)
fs.copy(
    config="fixtures/fs/copy-config.yaml",
    target_dir=Path("tmp/copy-target")
)

[2024-11-19T23:14:42]     INFO Validating config against internal schema: files-to-stage
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO File copies: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO File copies: Checking requirements
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-target/file1-copy.nml: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-target/file1-copy.nml: Checking requirements
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-target/file1-copy.nml: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-target/file1-copy.nml: Executing
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-target/file1-copy.nml: Final state: Ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file2.txt -> tmp/copy-target/data/file2-copy.txt: Initial state: Not Ready
[2024-11-19

True

Examining the target directory, we can see that the copies of the files have been made with their specified names and in their specified directories.

In [5]:
%%bash
tree tmp/copy-target

tmp/copy-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

1 directory, 3 files


### Failing to copy

A configuration can be provided as a dictionary instead as this example demonstrates. However, `missing-file.nml` does not exist. The function provides a warning and returns `False`.

In [6]:
fs.copy(
    config={"missing-copy.nml":"fixtures/fs/missing-file.nml"},
    target_dir="tmp/copy-target"
)

[2024-11-19T23:14:42]     INFO Validating config against internal schema: files-to-stage
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO File copies: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO File copies: Checking requirements
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/missing-file.nml -> tmp/copy-target/missing-copy.nml: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/missing-file.nml -> tmp/copy-target/missing-copy.nml: Checking requirements
[2024-11-19T23:14:42]  WARNING File fixtures/fs/missing-file.nml: State: Not Ready (external asset)
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/missing-file.nml -> tmp/copy-target/missing-copy.nml: Requirement(s) not ready
[2024-11-19T23:14:42]  WARNING Copy fixtures/fs/missing-file.nml -> tmp/copy-target/missing-copy.nml: Final state: Not Ready
[2024-11-19T23:14:42]  WARNING File copies: Final state: Not Ready


False

The missing copy does not appear in the target directory.

In [7]:
%%bash
tree tmp/copy-target

tmp/copy-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

1 directory, 3 files


### Using the `keys` parameter<!--copy-->

Consider the following configuration, in which the destination/source mapping is not located at the top level of the configuration:

In [8]:
%%bash
cat fixtures/fs/copy-keys-config.yaml

files:
  to:
    copy:
      file1-copy.nml: fixtures/fs/file1.nml
      data/file2-copy.txt: fixtures/fs/file2.txt
      data/file3-copy.csv: fixtures/fs/file3.csv


Without additional information, `copy()` would raise a `UWConfigError` given this configuration. However, the list of keys leading to the destination/source mapping can be provided with the `keys` parameter:

In [9]:
rmtree("tmp/copy-keys-target", ignore_errors=True)
fs.copy(
    config="fixtures/fs/copy-keys-config.yaml",
    target_dir="tmp/copy-keys-target",
    keys=["files","to","copy"]
)

[2024-11-19T23:14:42]     INFO Validating config against internal schema: files-to-stage
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO File copies: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO File copies: Checking requirements
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-keys-target/file1-copy.nml: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-keys-target/file1-copy.nml: Checking requirements
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-keys-target/file1-copy.nml: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-keys-target/file1-copy.nml: Executing
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copy-keys-target/file1-copy.nml: Final state: Ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file2.txt -> tmp/copy-keys-target/data/file2-copy.txt: Initia

True

With this information provided, the copy is successful.

In [10]:
%%bash
tree tmp/copy-keys-target

tmp/copy-keys-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

1 directory, 3 files


### Using the `Copier` class

An alternative to using `copy()` is to instantiate a `Copier` object , then call its `go()` method.

In [11]:
help(fs.Copier)

Help on class Copier in module uwtools.fs:

class Copier(FileStager)
 |  Copier(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False) -> None
 |
 |  Stage files by copying.
 |
 |  Method resolution order:
 |      Copier
 |      FileStager
 |      Stager
 |      abc.ABC
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  go(self)
 |      Copy files.
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  __abstractmethods__ = frozenset()
 |
 |  ----------------------------------------------------------------------
 |  Methods inherited from Stager:
 |
 |  __init__(self, config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.d

A `Copier` object is instantiated using the same parameters as `copy()`, but copying is not performed until `Copier.go()` is called.

In [12]:
rmtree("tmp/copier-target", ignore_errors=True)
copier = fs.Copier(
    config="fixtures/fs/copy-config.yaml",
    target_dir="tmp/copier-target"
)
copier.go()

[2024-11-19T23:14:42]     INFO Validating config against internal schema: files-to-stage
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO File copies: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO File copies: Checking requirements
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copier-target/file1-copy.nml: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copier-target/file1-copy.nml: Checking requirements
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copier-target/file1-copy.nml: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copier-target/file1-copy.nml: Executing
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file1.nml -> tmp/copier-target/file1-copy.nml: Final state: Ready
[2024-11-19T23:14:42]     INFO Copy fixtures/fs/file2.txt -> tmp/copier-target/data/file2-copy.txt: Initial state: Not Ready

[Asset(ref=PosixPath('tmp/copier-target/file1-copy.nml'), ready=<bound method Path.is_file of PosixPath('tmp/copier-target/file1-copy.nml')>),
 Asset(ref=PosixPath('tmp/copier-target/data/file2-copy.txt'), ready=<bound method Path.is_file of PosixPath('tmp/copier-target/data/file2-copy.txt')>),
 Asset(ref=PosixPath('tmp/copier-target/data/file3-copy.csv'), ready=<bound method Path.is_file of PosixPath('tmp/copier-target/data/file3-copy.csv')>)]

Once `Copier.go()` is called, copies are created in the same way as they would have with `copy()`.

In [13]:
%%bash
tree tmp/copier-target

tmp/copier-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

1 directory, 3 files


## Linking files

The `link()` function creates symbolic links to files, automatically creating parent directories as needed.

In [14]:
help(fs.link)

Help on function link in module uwtools.api.fs:

link(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False, stdin_ok: bool = False) -> bool
    Link files.

    :param config: YAML-file path, or ``dict`` (read ``stdin`` if missing or ``None``).
    :param target_dir: Path to target directory.
    :param cycle: A datetime object to make available for use in the config.
    :param leadtime: A timedelta object to make available for use in the config.
    :param keys: YAML keys leading to file dst/src block.
    :param dry_run: Do not link files.
    :param stdin_ok: OK to read from ``stdin``?
    :return: ``True`` if all links were created.



Links to be created are specified by a mapping from keys destination-pathname keys to source-pathname values, either in a YAML file or a Python ``dict``.

In [15]:
%%bash
cat fixtures/fs/link-config.yaml

file1-link.nml: fixtures/fs/file1.nml
file2-link.txt: fixtures/fs/file2.txt
data/file3-link.csv: fixtures/fs/file3.csv


With these instructions, `link()` creates a symbolic link of each given file with the given name and in the given subdirectory. Links are created in the directory indicated by `target_dir`. Paths can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Any directories in the targeted paths will be created if they don't already exist. `True` is returned upon a successful run.

In [16]:
rmtree("tmp/link-target", ignore_errors=True)
fs.link(
    config=Path("fixtures/fs/link-config.yaml"),
    target_dir="tmp/link-target"
)

[2024-11-19T23:14:42]     INFO Validating config against internal schema: files-to-stage
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO File links: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO File links: Checking requirements
[2024-11-19T23:14:42]     INFO Link tmp/link-target/file1-link.nml -> fixtures/fs/file1.nml: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Link tmp/link-target/file1-link.nml -> fixtures/fs/file1.nml: Checking requirements
[2024-11-19T23:14:42]     INFO Link tmp/link-target/file1-link.nml -> fixtures/fs/file1.nml: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Link tmp/link-target/file1-link.nml -> fixtures/fs/file1.nml: Executing
[2024-11-19T23:14:42]     INFO Link tmp/link-target/file1-link.nml -> fixtures/fs/file1.nml: Final state: Ready
[2024-11-19T23:14:42]     INFO Link tmp/link-target/file2-link.txt -> fixtures/fs/file2.txt: Initial state: Not Ready
[2024-11-19T23:14:

True

Examining the target directory, we can see that the links have been created with their specified names and in their specified directories.

In [17]:
%%bash
tree tmp/link-target

tmp/link-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/file3.csv
├── file1-link.nml -> ../../fixtures/fs/file1.nml
└── file2-link.txt -> ../../fixtures/fs/file2.txt

1 directory, 3 files


### Failing to link

A configuration can be provided as a dictionary instead as this example demonstrates. However, `missing-file.nml` does not exist. The function provides a warning and returns `False`.

In [18]:
fs.link(
    config={"missing-link.nml":"fixtures/fs/missing-file.nml"},
    target_dir="tmp/link-target"
)

[2024-11-19T23:14:42]     INFO Validating config against internal schema: files-to-stage
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO File links: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO File links: Checking requirements
[2024-11-19T23:14:42]     INFO Link tmp/link-target/missing-link.nml -> fixtures/fs/missing-file.nml: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Link tmp/link-target/missing-link.nml -> fixtures/fs/missing-file.nml: Checking requirements
[2024-11-19T23:14:42]  WARNING Filesystem item fixtures/fs/missing-file.nml: State: Not Ready (external asset)
[2024-11-19T23:14:42]     INFO Link tmp/link-target/missing-link.nml -> fixtures/fs/missing-file.nml: Requirement(s) not ready
[2024-11-19T23:14:42]  WARNING Link tmp/link-target/missing-link.nml -> fixtures/fs/missing-file.nml: Final state: Not Ready
[2024-11-19T23:14:42]  WARNING File links: Final state: Not Ready


False

The missing link does not appear in the target directory.

In [19]:
%%bash
tree tmp/link-target

tmp/link-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/file3.csv
├── file1-link.nml -> ../../fixtures/fs/file1.nml
└── file2-link.txt -> ../../fixtures/fs/file2.txt

1 directory, 3 files


### Using the `keys` parameter <!--link-->

Consider the following configuration, in which the destination/source mapping is not located at the top level of the configuration:

In [20]:
%%bash
cat fixtures/fs/link-keys-config.yaml

files:
  to:
    link:
      file1-link.nml: fixtures/fs/file1.nml
      file2-link.txt: fixtures/fs/file2.txt
      data/file3-link.csv: fixtures/fs/file3.csv


Without additional information, `link()` would raise a `UWConfigError` given this configuration. However, the list of keys leading to the destination/source mapping can be provided with the `keys` parameter:

In [21]:
rmtree("tmp/link-keys-target", ignore_errors=True)
fs.link(
    config="fixtures/fs/link-keys-config.yaml",
    target_dir="tmp/link-keys-target",
    keys=["files","to","link"]
)

[2024-11-19T23:14:42]     INFO Validating config against internal schema: files-to-stage
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO File links: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO File links: Checking requirements
[2024-11-19T23:14:42]     INFO Link tmp/link-keys-target/file1-link.nml -> fixtures/fs/file1.nml: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Link tmp/link-keys-target/file1-link.nml -> fixtures/fs/file1.nml: Checking requirements
[2024-11-19T23:14:42]     INFO Link tmp/link-keys-target/file1-link.nml -> fixtures/fs/file1.nml: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Link tmp/link-keys-target/file1-link.nml -> fixtures/fs/file1.nml: Executing
[2024-11-19T23:14:42]     INFO Link tmp/link-keys-target/file1-link.nml -> fixtures/fs/file1.nml: Final state: Ready
[2024-11-19T23:14:42]     INFO Link tmp/link-keys-target/file2-link.txt -> fixtures/fs/file2.txt: Initial state

True

With this information provided, the links are successfully created.

In [22]:
%%bash
tree tmp/link-keys-target

tmp/link-keys-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/file3.csv
├── file1-link.nml -> ../../fixtures/fs/file1.nml
└── file2-link.txt -> ../../fixtures/fs/file2.txt

1 directory, 3 files


### Using the `Linker` class

An alternative to using `link()` is to instantiate a `Linker` object , then call its `go()` method.

In [23]:
help(fs.Linker)

Help on class Linker in module uwtools.fs:

class Linker(FileStager)
 |  Linker(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False) -> None
 |
 |  Stage files by linking.
 |
 |  Method resolution order:
 |      Linker
 |      FileStager
 |      Stager
 |      abc.ABC
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  go(self)
 |      Link files.
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  __abstractmethods__ = frozenset()
 |
 |  ----------------------------------------------------------------------
 |  Methods inherited from Stager:
 |
 |  __init__(self, config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.d

A `Linker` object is instantiated using the same parameters as `link()`, but links are not created until `Linker.go()` is called.

In [24]:
rmtree("tmp/linker-target", ignore_errors=True)
linker = fs.Linker(
    config="fixtures/fs/link-config.yaml",
    target_dir="tmp/linker-target"
)
linker.go()

[2024-11-19T23:14:42]     INFO Validating config against internal schema: files-to-stage
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO File links: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO File links: Checking requirements
[2024-11-19T23:14:42]     INFO Link tmp/linker-target/file1-link.nml -> fixtures/fs/file1.nml: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Link tmp/linker-target/file1-link.nml -> fixtures/fs/file1.nml: Checking requirements
[2024-11-19T23:14:42]     INFO Link tmp/linker-target/file1-link.nml -> fixtures/fs/file1.nml: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Link tmp/linker-target/file1-link.nml -> fixtures/fs/file1.nml: Executing
[2024-11-19T23:14:42]     INFO Link tmp/linker-target/file1-link.nml -> fixtures/fs/file1.nml: Final state: Ready
[2024-11-19T23:14:42]     INFO Link tmp/linker-target/file2-link.txt -> fixtures/fs/file2.txt: Initial state: Not Ready
[2024-

[Asset(ref=PosixPath('tmp/linker-target/file1-link.nml'), ready=<bound method Path.exists of PosixPath('tmp/linker-target/file1-link.nml')>),
 Asset(ref=PosixPath('tmp/linker-target/file2-link.txt'), ready=<bound method Path.exists of PosixPath('tmp/linker-target/file2-link.txt')>),
 Asset(ref=PosixPath('tmp/linker-target/data/file3-link.csv'), ready=<bound method Path.exists of PosixPath('tmp/linker-target/data/file3-link.csv')>)]

Once `Linker.go()` is called, links are created in the same way as they would have with `link()`.

In [25]:
%%bash
tree tmp/linker-target

tmp/linker-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/file3.csv
├── file1-link.nml -> ../../fixtures/fs/file1.nml
└── file2-link.txt -> ../../fixtures/fs/file2.txt

1 directory, 3 files


## Creating directories

The `makedirs()` function creates directories.

In [26]:
help(fs.makedirs)

Help on function makedirs in module uwtools.api.fs:

makedirs(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False, stdin_ok: bool = False) -> bool
    Make directories.

    :param config: YAML-file path, or ``dict`` (read ``stdin`` if missing or ``None``).
    :param target_dir: Path to target directory.
    :param cycle: A datetime object to make available for use in the config.
    :param leadtime: A timedelta object to make available for use in the config.
    :param keys: YAML keys leading to file dst/src block.
    :param dry_run: Do not link files.
    :param stdin_ok: OK to read from ``stdin``?
    :return: ``True`` if all directories were made.



Directories to be created are specified by either a configuration YAML file or a Python ``dict``. A `makedirs` key must be included with a list of directories to create as its value.

In [27]:
%%bash
cat fixtures/fs/dir-config.yaml

makedirs:
  - foo
  - bar/baz


With these instructions, `makedirs()` creates each directory in the list within the directory indicated by `target_dir`. Paths can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. `True` is returned upon a successful run.

In [28]:
rmtree("tmp/dir-target", ignore_errors=True)
fs.makedirs(
    config="fixtures/fs/dir-config.yaml",
    target_dir=Path("tmp/dir-target")
)

[2024-11-19T23:14:42]     INFO Validating config against internal schema: makedirs
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO Directories: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Directories: Checking requirements
[2024-11-19T23:14:42]     INFO Directory tmp/dir-target/foo: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Directory tmp/dir-target/foo: Checking requirements
[2024-11-19T23:14:42]     INFO Directory tmp/dir-target/foo: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Directory tmp/dir-target/foo: Executing
[2024-11-19T23:14:42]     INFO Directory tmp/dir-target/foo: Final state: Ready
[2024-11-19T23:14:42]     INFO Directory tmp/dir-target/bar/baz: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Directory tmp/dir-target/bar/baz: Checking requirements
[2024-11-19T23:14:42]     INFO Directory tmp/dir-target/bar/baz: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Direct

True

Examining the target directory, we can see that the directories have been created with their specified names.

In [29]:
%%bash
tree tmp/dir-target

tmp/dir-target
├── bar
│   └── baz
└── foo

3 directories, 0 files


### Using the `keys` parameter  <!--dir-->

Consider the following configuration, in which the destination/source mapping is not located at the top level of the configuration:

In [30]:
%%bash
cat fixtures/fs/dir-keys-config.yaml

path:
  to:
    dirs:
      makedirs:
        - foo/bar
        - baz


Without additional information, `makedirs()` would raise a `UWConfigError` given this configuration. However, the list of keys leading to the destination/source mapping can be provided with the `keys` parameter:

In [31]:
rmtree("tmp/dir-keys-target", ignore_errors=True)
fs.makedirs(
    config="fixtures/fs/dir-keys-config.yaml",
    target_dir="tmp/dir-keys-target",
    keys=["path","to","dirs"]
)

[2024-11-19T23:14:42]     INFO Validating config against internal schema: makedirs
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO Directories: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Directories: Checking requirements
[2024-11-19T23:14:42]     INFO Directory tmp/dir-keys-target/foo/bar: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Directory tmp/dir-keys-target/foo/bar: Checking requirements
[2024-11-19T23:14:42]     INFO Directory tmp/dir-keys-target/foo/bar: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Directory tmp/dir-keys-target/foo/bar: Executing
[2024-11-19T23:14:42]     INFO Directory tmp/dir-keys-target/foo/bar: Final state: Ready
[2024-11-19T23:14:42]     INFO Directory tmp/dir-keys-target/baz: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Directory tmp/dir-keys-target/baz: Checking requirements
[2024-11-19T23:14:42]     INFO Directory tmp/dir-keys-target/baz: Requiremen

True

With this information provided, the directories are successfully created.

In [32]:
%%bash
tree tmp/dir-keys-target

tmp/dir-keys-target
├── baz
└── foo
    └── bar

3 directories, 0 files


### Using the `MakeDirs` class

An alternative to using `makedirs()` is to instantiate a `MakeDirs` object , then call its `go()` method.

In [33]:
help(fs.MakeDirs)

Help on class MakeDirs in module uwtools.fs:

class MakeDirs(Stager)
 |  MakeDirs(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False) -> None
 |
 |  Make directories.
 |
 |  Method resolution order:
 |      MakeDirs
 |      Stager
 |      abc.ABC
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  go(self)
 |      Make directories.
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  __abstractmethods__ = frozenset()
 |
 |  ----------------------------------------------------------------------
 |  Methods inherited from Stager:
 |
 |  __init__(self, config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None

A `MakeDirs` object is instantiated using the same parameters as `makedirs()`, but directories are not created until `MakeDirs.go()` is called.

In [34]:
rmtree("tmp/makedirs-target", ignore_errors=True)
dirs_stager = fs.MakeDirs(
    config="fixtures/fs/dir-config.yaml",
    target_dir="tmp/makedirs-target"
)
dirs_stager.go()

[2024-11-19T23:14:42]     INFO Validating config against internal schema: makedirs
[2024-11-19T23:14:42]     INFO 0 UW schema-validation errors found in fs config
[2024-11-19T23:14:42]     INFO Directories: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Directories: Checking requirements
[2024-11-19T23:14:42]     INFO Directory tmp/makedirs-target/foo: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Directory tmp/makedirs-target/foo: Checking requirements
[2024-11-19T23:14:42]     INFO Directory tmp/makedirs-target/foo: Requirement(s) ready
[2024-11-19T23:14:42]     INFO Directory tmp/makedirs-target/foo: Executing
[2024-11-19T23:14:42]     INFO Directory tmp/makedirs-target/foo: Final state: Ready
[2024-11-19T23:14:42]     INFO Directory tmp/makedirs-target/bar/baz: Initial state: Not Ready
[2024-11-19T23:14:42]     INFO Directory tmp/makedirs-target/bar/baz: Checking requirements
[2024-11-19T23:14:42]     INFO Directory tmp/makedirs-target/bar/baz: Requirement(s) rea

[Asset(ref=PosixPath('tmp/makedirs-target/foo'), ready=<bound method Path.is_dir of PosixPath('tmp/makedirs-target/foo')>),
 Asset(ref=PosixPath('tmp/makedirs-target/bar/baz'), ready=<bound method Path.is_dir of PosixPath('tmp/makedirs-target/bar/baz')>)]

Once `MakeDirs.go()` is called, directories are created in the same way as they would have with `makedirs()`.

In [35]:
%%bash
tree tmp/makedirs-target

tmp/makedirs-target
├── bar
│   └── baz
└── foo

3 directories, 0 files
